In [1]:
from __future__ import print_function
import os
import numpy as np
import sys
import subprocess
import tarfile
import xml.etree.ElementTree as ET
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from yaafelib import *

In [2]:
class FeatureExtractor:
    def __init__(self, audiofile):
        self.audiofile = audiofile
        self.temporal_centroid = None  # 1: Temporal Centroid
        self.spectral_centroid = None  # 2: Spectral Centroid average value
        self.ase_per_band_avg = []  # 4-37: Audio Spectrum Envelope (ASE) average values in 34 frequency bands
        self.ase_avg = None  # 38: ASE average value (averaged for all frequency bands)
        self.ase_per_band_var = []  # 39-72: ASE variance values in 34 frequency bands
        self.ase_var_avg = None  # 73: averaged ASE variance parameters
        self.centroid_avg = None  # 74: Audio Spectrum Centroid – average
        self.centroid_var = None  # 75: Audio Spectrum Centroid – variance
        self.spread_avg = None  # 76: Audio Spectrum Spread – average
        self.spread_var = None  # 77: Audio Spectrum Spread – variance
        self.sfm_per_band_avg = []  # 78-101: Spectral Flatness Measure (SFM) average values for 24 frequency bands
        self.sfm_avg = None  # 102: SFM average value (averaged for all frequency bands)
        self.sfm_per_band_var = []  # 103-126: Spectral Flatness Measure (SFM) variance values for 24 frequency bands
        self.sfm_var_avg = None  # 127: averaged SFM variance parameters
        self.mfcc = []  # 128-147: 20 first mel cepstral coefficients average values
        self.harmonic_spectral_centroid = None
        self.harmonic_spectral_deviation = None
        self.harmonic_spectral_spread = None
        self.harmonic_spectral_variation = None
        self.audio_fundamental_frequency_avg = None
        self.audio_fundamental_frequency_var = None
        self.harmonic_ratio_avg = None
        self.harmonic_ratio_var = None
        self.upper_limit_harmonicity_avg = None
        self.upper_limit_harmonicity_var = None
        self.audio_power_avg = None
        self.audio_power_var = None
        self.log_attack_time = None
        self.fv = []
        self.extract_features()
        self.export_features()

    def export_features(self):
        self.fv.append(self.temporal_centroid)
        self.fv.append(self.spectral_centroid)
        self.fv.extend(filter(lambda a: a != 0, self.ase_per_band_avg))
        self.fv.append(self.ase_avg)
        self.fv.extend(filter(lambda a: a != 0, self.ase_per_band_var))
        self.fv.append(self.ase_var_avg)
        self.fv.append(self.centroid_avg)
        self.fv.append(self.centroid_var)
        self.fv.append(self.spread_avg)
        self.fv.append(self.spread_var)
        self.fv.extend(self.sfm_per_band_avg)
        self.fv.append(self.sfm_avg)
        self.fv.extend(self.sfm_per_band_var)
        self.fv.append(self.sfm_var_avg)
        self.fv.extend(self.mfcc.tolist()[0])
        self.fv.append(self.harmonic_spectral_centroid)
        self.fv.append(self.harmonic_spectral_deviation)
        self.fv.append(self.harmonic_spectral_spread)
        self.fv.append(self.harmonic_spectral_variation)
        self.fv.append(self.audio_fundamental_frequency_avg)
        self.fv.append(self.audio_fundamental_frequency_var)
        self.fv.append(self.audio_power_avg)
        self.fv.append(self.audio_power_var)
        self.fv.append(self.log_attack_time)

    def extract_features(self):
        if os.path.exists(self.audiofile):
            print('Getting features from ' + self.audiofile)
        else:
            raise Exception('File ' + self.audiofile + ' not found')

        self.extract_mpeg7_features()
        self.extract_mfcc()

    def extract_mpeg7_features(self):
        ns = {'xmlns': 'urn:mpeg:mpeg7:schema:2001',
              'mpeg7': 'urn:mpeg:mpeg7:schema:2001',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
              'xsi:schemaLocation': 'urn:mpeg:mpeg7:schema:2001 http://mpeg7audioenc.sourceforge.net/mpeg7audioenc.xsd'}

        result = subprocess.check_output(['java', '-jar', 'MPEG7AudioEnc.jar', self.audiofile, 'mpeg7config.xml'])
        root = ET.fromstring(result)

        self.extract_scalar_features(root, ns)
        self.extract_vector_features(root, ns)
        self.extract_matrix_features(root, ns)

    def extract_scalar_features(self, root, ns):

        self.temporal_centroid = self.parse_xml_scalar(root,
                                                       ".//mpeg7:AudioDescriptor[@xsi:type='TemporalCentroidType']", ns)
        self.spectral_centroid = self.parse_xml_scalar(root,
                                                       ".//mpeg7:AudioDescriptor[@xsi:type='SpectralCentroidType']", ns)
        self.harmonic_spectral_centroid = self.parse_xml_scalar(root,
                                                                ".//mpeg7:AudioDescriptor[@xsi:type='HarmonicSpectralCentroidType']",
                                                                ns)
        self.harmonic_spectral_deviation = self.parse_xml_scalar(root,
                                                                 ".//mpeg7:AudioDescriptor[@xsi:type='HarmonicSpectralDeviationType']",
                                                                 ns)
        self.harmonic_spectral_spread = self.parse_xml_scalar(root,
                                                              ".//mpeg7:AudioDescriptor[@xsi:type='HarmonicSpectralSpreadType']",
                                                              ns)
        self.harmonic_spectral_variation = self.parse_xml_scalar(root,
                                                                 ".//mpeg7:AudioDescriptor[@xsi:type='HarmonicSpectralVariationType']",
                                                                 ns)
        self.log_attack_time = self.parse_xml_scalar(root, ".//mpeg7:AudioDescriptor[@xsi:type='LogAttackTimeType']",
                                                     ns)

    def extract_vector_features(self, root, ns):
        audio_spectrum_centroid_values = self.parse_xml_vector(root,
                                                               ".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumCentroidType']",
                                                               ns)
        self.centroid_avg = np.mean(audio_spectrum_centroid_values)
        self.centroid_var = np.var(audio_spectrum_centroid_values)

        audio_spectrum_spread_values = self.parse_xml_vector(root,
                                                             ".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumSpreadType']",
                                                             ns)
        self.spread_avg = np.mean(audio_spectrum_spread_values)
        self.spread_var = np.var(audio_spectrum_spread_values)

        audio_fundamental_frequency_values = self.parse_xml_vector(root,
                                                                   ".//mpeg7:AudioDescriptor[@xsi:type='AudioFundamentalFrequencyType']",
                                                                   ns)
        self.audio_fundamental_frequency_avg = np.mean(audio_fundamental_frequency_values)
        self.audio_fundamental_frequency_var = np.var(audio_fundamental_frequency_values)

        harmonic_ratio_values = self.parse_xml_vector(root, ".//mpeg7:HarmonicRatio", ns)
        self.harmonic_ratio_avg = np.mean(harmonic_ratio_values)
        self.harmonic_ratio_var = np.var(harmonic_ratio_values)

        upper_limit_harmonicity_values = self.parse_xml_vector(root, ".//mpeg7:UpperLimitOfHarmonicity", ns)
        self.upper_limit_harmonicity_avg = np.mean(upper_limit_harmonicity_values)
        self.upper_limit_harmonicity_var = np.var(upper_limit_harmonicity_values)

        audio_power_values = self.parse_xml_vector(root, ".//mpeg7:AudioDescriptor[@xsi:type='AudioPowerType']", ns)
        self.audio_power_avg = np.mean(audio_power_values)
        self.audio_power_var = np.var(audio_power_values)

    def extract_matrix_features(self, root, ns):
        envelope_values = self.parse_2d_xml_vector(root,
                                                   ".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumEnvelopeType']",
                                                   ns)
        self.ase_per_band_avg = [np.mean(band) for band in envelope_values]
        self.ase_avg = np.mean(self.ase_per_band_avg)
        self.ase_per_band_var = [np.var(band) for band in envelope_values]
        self.ase_var_avg = np.mean(self.ase_per_band_var)

        flatness_values = self.parse_2d_xml_vector(root,
                                                   ".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumFlatnessType']",
                                                   ns)
        self.sfm_per_band_avg = [np.mean(band) for band in flatness_values]
        self.sfm_avg = np.mean(self.sfm_per_band_avg)
        self.sfm_per_band_var = [np.var(band) for band in flatness_values]
        self.sfm_var_avg = np.mean(self.sfm_per_band_var)

    def parse_xml_scalar(self, root, xml_selector, ns):
        element = root.find(xml_selector, ns)
        return float(element.find(".//mpeg7:Scalar", ns).text)

    def parse_xml_vector(self, root, xml_selector, ns):
        element = root.find(xml_selector, ns)
        values_splitted = element.find(".//mpeg7:Raw", ns).text.split()
        return map((lambda x: float(x)), values_splitted)

    def parse_2d_xml_vector(self, root, xml_selector, ns):
        element = root.find(xml_selector, ns)
        values_string = element.find(".//mpeg7:Raw", ns).text
        values_splitted = [s.strip().split() for s in values_string.splitlines()]
        values = [map((lambda x: float(x)), value) for value in values_splitted]  # cast to float
        transposed_values = list(map(list, zip(*values)))  # transpose matrix to have 1 long vector per 1 band
        return transposed_values

    def extract_mfcc(self):
        fp = FeaturePlan(sample_rate=22050, normalize=1)
        fp.addFeature('mfcc: MFCC CepsNbCoeffs=20')
        df = fp.getDataFlow()
        engine = Engine()
        engine.load(df)
        afp = AudioFileProcessor()
        afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
        afp.processFile(engine, self.audiofile)
        engine.flush()
        feats = engine.readAllOutputs()
        self.mfcc = feats['mfcc']

    def get_features(self):
        return self.fv

In [4]:
root = os.path.join(os.getcwd(), 'data/genres')

genres = os.listdir(root)
mappings = dict(enumerate(genres))
mappings_rev = {v: k for k, v in mappings.items()}
dataset_xs = []
dataset_y = []

{'reggae': 5, 'classical': 0, 'country': 7, 'jazz': 8, 'metal': 2, 'pop': 9, 'disco': 3, 'hiphop': 4, 'rock': 6, 'blues': 1}


In [26]:
for folder in os.listdir(root):
    data_folder = os.path.join(root, folder)
    filenames = os.listdir(data_folder)
    i=0;
    for filename in filenames:
        path = os.path.join(data_folder, filename)
        genre = mappings_rev[folder]
        featureExtractor = FeatureExtractor(path)
        i+=1
        dataset_xs.append(featureExtractor.fv)
        dataset_y.append(genre)
        print(i)

Getting features from /app/audio_parser/data/genres/classical/classical.00015.au
1
Getting features from /app/audio_parser/data/genres/classical/classical.00061.au
2
Getting features from /app/audio_parser/data/genres/classical/classical.00012.au
3
Getting features from /app/audio_parser/data/genres/classical/classical.00050.au
4
Getting features from /app/audio_parser/data/genres/classical/classical.00098.au
5
Getting features from /app/audio_parser/data/genres/classical/classical.00086.au
6
Getting features from /app/audio_parser/data/genres/classical/classical.00002.au
7
Getting features from /app/audio_parser/data/genres/classical/classical.00099.au
8
Getting features from /app/audio_parser/data/genres/classical/classical.00054.au
9
Getting features from /app/audio_parser/data/genres/classical/classical.00090.au
10
Getting features from /app/audio_parser/data/genres/classical/classical.00060.au
11
Getting features from /app/audio_parser/data/genres/classical/classical.00091.au
12
G

In [37]:
print(dataset_y.__len__())

2031


In [32]:
def evens(dataset):
    return dataset[::2]


def odds(dataset):
    return dataset[1::2]



train_dataset_xs = evens(dataset_xs)
train_dataset_y = evens(dataset_y)
test_dataset_xs = odds(dataset_xs)
test_dataset_y = odds(dataset_y)

def randomize(dataset_xs, dataset_y):
    permutation = np.random.permutation(len(dataset_y))
    shuffled_dataset_xs = np.asarray(dataset_xs)[permutation]
    shuffled_dataset_y = np.asarray(dataset_y)[permutation]
    return shuffled_dataset_xs, shuffled_dataset_y


train_dataset_xs, train_dataset_y = randomize(train_dataset_xs, train_dataset_y)
test_dataset_xs, test_dataset_y = randomize(test_dataset_xs, test_dataset_y)

try:
    f = open('data/gztan.pickle', 'wb')
    save = {
        'train_dataset_xs': train_dataset_xs,
        'train_dataset_y': train_dataset_y,
        'test_dataset_xs': test_dataset_xs,
        'test_dataset_y': test_dataset_y,
        'mappings': mappings
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', 'data/gztan.pickle', ':', e)
    raise

298
